## Segmenting and Clustering Neighborhoods in Toronto part :1

In [1]:
import requests
import lxml.html as lh
import pandas as pd

#### 1-Downloading data and  Creating the pandas data frame :

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
doc = lh.fromstring(page.content)
elements = doc.xpath('//tr')

In [3]:
elements = doc.xpath('//tr')
col=[]
i=0
for t in elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postcode"
2:"Borough"
3:"Neighbourhood
"


In [4]:
for j in range(1,len(elements)):
    T=elements[j]
    
    if len(T)!=3:
        break
    i=0
    
    for t in T.iterchildren():
        data=t.text_content() 
        if i>0:
            try:
                data=int(data)
            except:
                pass
        col[i][1].append(data)
        i+=1

In [5]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [6]:
df.columns = ['Borough', 'Neighbourhood','Postcode']

col = df.columns.tolist()
col

col = col[-1:] + col[:-1]

df = df[col]

df.head()

,Postcode,Borough,Neighbourhood
0,Not assigned\n,M1A,Not assigned
1,Not assigned\n,M2A,Not assigned
2,Parkwoods\n,M3A,North York
3,Victoria Village\n,M4A,North York
4,Harbourfront\n,M5A,Downtown Toronto


#### 2-correcting the  string format  in the Borough column and  Ignoring cells with a borough that is Not assigned:

In [7]:
df = df.replace('\n',' ', regex=True)
df.head()

,Postcode,Borough,Neighbourhood
0,Not assigned,M1A,Not assigned
1,Not assigned,M2A,Not assigned
2,Parkwoods,M3A,North York
3,Victoria Village,M4A,North York
4,Harbourfront,M5A,Downtown Toronto


In [8]:
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)
df = df.reset_index(drop=True)
df.head(10)

,Postcode,Borough,Neighbourhood
0,Not assigned,M1A,Not assigned
1,Not assigned,M2A,Not assigned
2,Parkwoods,M3A,North York
3,Victoria Village,M4A,North York
4,Harbourfront,M5A,Downtown Toronto
5,Lawrence Heights,M6A,North York
6,Lawrence Manor,M6A,North York
7,Queen's Park,M7A,Downtown Toronto
8,Not assigned,M8A,Not assigned
9,Islington Avenue,M9A,Etobicoke


#### 3-Grouping Neighbourhoods by similar Postcode and Borough  :

In [9]:
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
df.columns = ['Postcode','Borough','Neighbourhood']
df.head(10)

,Postcode,Borough,Neighbourhood
0,Adelaide,M5H,Downtown Toronto
1,Agincourt,M1S,Scarborough
2,Agincourt North,M1V,Scarborough
3,Albion Gardens,M9V,Etobicoke
4,Alderwood,M8W,Etobicoke
5,Bathurst Manor,M3H,North York
6,Bathurst Quay,M5V,Downtown Toronto
7,Bayview Village,M2K,North York
8,Beaumond Heights,M9V,Etobicoke
9,Bedford Park,M5M,North York


In [10]:
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']
df.head()

,Postcode,Borough,Neighbourhood
0,Adelaide,M5H,Downtown Toronto
1,Agincourt,M1S,Scarborough
2,Agincourt North,M1V,Scarborough
3,Albion Gardens,M9V,Etobicoke
4,Alderwood,M8W,Etobicoke


In [11]:
df.shape

(287, 3)

In [12]:
df.to_csv(r'df_canada.csv')